In [3]:
import pandas as pd
import numpy as np 
from copy import deepcopy
from util.dataloader import cached_bison_data, load_pump_curves, cached_site_info, fetch_bison_data
from util.preprocessing import *
from util.calculations import *
from util.format_calculations import *
import plotly.express as px

# fetch data
# filepath = "/Users/audreyder/Neuralix/syncdatabase_2025-02-10.csv"
# data = fetch_bison_data(filepath)

# load data
filepath = "/Users/audreyder/Neuralix/syncdatabase_2025-02-10.csv"
data = cached_bison_data(filepath) 

# filter data / preprocess
data = threshold_filtering(data) 
# load pump curve data
site_ids = [33404, 33467, 57740, 33614]
site_pump_curves = load_pump_curves(site_ids) # dictionary of dataframes // FUTURE PROOF NOTE: eventually, there will be many sites with many pumps, not just four sites with one pump each
sites_info = cached_site_info(dict=True)

Data columns: Index(['site_id', 'pump_id', 'timestamp', 'facility_name', 'vibration',
       'thrust temperature', 'suction pressure', 'discharge pressure',
       'flow rate', 'frequency', 'amps', 'ampsa', 'ampsb', 'ampsc', 'volts',
       'voltsa', 'voltsb', 'voltsc', 'meter total'],
      dtype='object')
Number of rows: 261210
Earliest timestamp: 2024-11-12 16:55:10
Latest timestamp: 2025-02-10 16:53:03


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/preprocessing.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[frequency_int_column] = df[frequency_float_column].round().astype('Int64')


In [ ]:
kpis_all_sites = {site_id:None for site_id in site_ids}
site_pump_curves = load_pump_curves(site_ids)
for site_id in site_ids: # for each site...
    site_data = deepcopy(data[data['site_id']==site_id]) # Looks at individual site data
    pump_curve_data = site_pump_curves[site_id]
    site_data, site_pump_kpis, sampling_interval = compute_kpis_for_sites(site_data, site_id,kpis=['kWh/BBL','Flow Rate','perc_from_BEP','norm_perc_from_BEP'],site_pump_curves=pump_curve_data,sites_info=sites_info)
    site_name = sites_info[site_id]['site_name']
    kpis_all_sites[site_id] = site_pump_kpis

/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_time_info['timestamp_datetime'] = pd.to_datetime(site_time_info)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:309: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invalid_prev = invalid.shift(1).fillna(False)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:311: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

Recorded frequencies outside calibration range (KPI is not normalized): {39, 40, 42, 45, 57}
Interpolated frequencies within calibration range (KPI normalization is approximated): {47, 49, 51, 53, 55}
Normalized frequencies: {46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56}


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_time_info['timestamp_datetime'] = pd.to_datetime(site_time_info)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:309: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invalid_prev = invalid.shift(1).fillna(False)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:311: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

Recorded frequencies outside calibration range (KPI is not normalized): {37, 39, 40, 41, 42, 43, 44, 45, 46, 58, 59}
Interpolated frequencies within calibration range (KPI normalization is approximated): {48, 50, 52, 54, 56}
Normalized frequencies: {47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57}


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_time_info['timestamp_datetime'] = pd.to_datetime(site_time_info)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:309: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invalid_prev = invalid.shift(1).fillna(False)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:311: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

Recorded frequencies outside calibration range (KPI is not normalized): {42, 43}
Interpolated frequencies within calibration range (KPI normalization is approximated): {45, 46, 48, 50, 52, 54, 56, 58}
Normalized frequencies: {44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59}
No intersection found between speed line 65 and efficiency line Min
Missing intersection points for speed line 65, skipping


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_time_info['timestamp_datetime'] = pd.to_datetime(site_time_info)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:309: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invalid_prev = invalid.shift(1).fillna(False)
/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/calculations.py:311: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

Recorded frequencies outside calibration range (KPI is not normalized): {59, 60, 45, 46}
Interpolated frequencies within calibration range (KPI normalization is approximated): {49, 51, 53, 55, 57}
Normalized frequencies: {48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58}


In [7]:
df_all, game_kpi = kpi_charts(kpis_all_sites, site_ids, sites_info, kpis=['kWh/BBL','Flow Rate'],print_chart=True)

Union City 2 SWD


,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
47.0,9277.041,0.791,-98.055,98.055,-1.338,1.338,Union City 2 SWD
48.0,9343.039,0.822,-99.532,99.532,-2.793,2.793,Union City 2 SWD
49.0,10624.368,0.787,-88.626,88.626,-1.100,1.100,Union City 2 SWD
50.0,11209.843,0.795,-82.231,82.231,-1.103,1.103,Union City 2 SWD
51.0,11895.563,0.792,-74.460,74.460,-1.425,1.425,Union City 2 SWD
52.0,12693.939,0.793,-64.939,64.939,-1.642,1.642,Union City 2 SWD
53.0,13532.486,0.787,-54.910,54.910,-0.924,0.924,Union City 2 SWD
54.0,14114.580,0.802,-49.149,49.149,-1.177,1.177,Union City 2 SWD
55.0,15156.667,0.796,-36.157,36.157,-0.929,0.929,Union City 2 SWD
57.0,16916.667,0.802,-16.246,16.246,NaN,NaN,Union City 2 SWD


Siegrist SWD


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/format_calculations.py:80: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
47.0,12020.958,0.682,-37.070,37.070,-2.252,2.252,Siegrist SWD
48.0,12183.622,0.685,-38.080,38.080,-2.943,2.943,Siegrist SWD
49.0,12737.414,0.698,-33.384,33.384,-2.804,2.804,Siegrist SWD
50.0,13449.370,0.715,-26.405,26.405,-2.244,2.244,Siegrist SWD
51.0,13641.706,0.730,-27.207,27.207,-4.540,4.540,Siegrist SWD
52.0,14517.355,0.760,-17.824,17.824,-0.670,0.670,Siegrist SWD
53.0,14800.139,0.758,-17.422,17.422,-1.207,1.207,Siegrist SWD
54.0,15620.976,0.770,-8.999,8.999,0.906,0.906,Siegrist SWD
55.0,16010.592,0.772,-7.037,7.037,1.182,1.182,Siegrist SWD
56.0,16399.440,0.789,-4.839,4.839,0.759,0.759,Siegrist SWD


Canadian SWD


,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
42.0,8797.555,0.617,-68.059,68.059,NaN,NaN,Canadian SWD
43.0,9379.004,0.626,-62.560,62.560,NaN,NaN,Canadian SWD
44.0,10441.146,0.617,-49.968,49.968,0.505,0.505,Canadian SWD
45.0,10521.958,0.617,-52.153,52.153,-1.967,1.967,Canadian SWD
46.0,11586.816,0.645,-40.207,40.207,-0.141,0.141,Canadian SWD
47.0,11625.723,0.645,-43.291,43.291,-1.243,1.243,Canadian SWD
48.0,12387.424,0.682,-36.249,36.249,-1.120,1.120,Canadian SWD
49.0,12410.306,0.685,-39.570,39.570,-4.565,4.565,Canadian SWD
50.0,13787.503,0.709,-24.687,24.687,0.384,0.384,Canadian SWD
51.0,13818.616,0.712,-28.074,28.074,-1.613,1.613,Canadian SWD


Calumet SWD


,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
45.0,10994.628,0.596,-38.056,38.056,NaN,NaN,Calumet SWD
46.0,11451.400,0.612,-34.478,34.478,NaN,NaN,Calumet SWD
48.0,11780.576,0.670,-36.639,36.639,-2.880,2.880,Calumet SWD
49.0,12193.108,0.686,-33.774,33.774,-2.213,2.213,Calumet SWD
50.0,10537.047,0.787,-62.313,62.313,-12.017,12.017,Calumet SWD
51.0,12086.107,0.764,-42.164,42.164,-2.738,2.738,Calumet SWD
52.0,11503.003,0.815,-54.064,54.064,-3.050,3.050,Calumet SWD
53.0,13477.875,0.782,-28.432,28.432,-2.928,2.928,Calumet SWD
54.0,13095.556,0.827,-37.222,37.222,-53.864,53.864,Calumet SWD
55.0,14188.429,0.837,-24.703,24.703,-9.879,9.879,Calumet SWD
